In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1.]
[1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-10 22:45:36,171	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-10_22-45-36_11704/logs.
2019-06-10 22:45:36,282	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:41351 to respond...
2019-06-10 22:45:36,423	WARNING services.py:761 -- Redis failed to start, retrying now.
2019-06-10 22:45:36,563	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:20466 to respond...
2019-06-10 22:45:36,573	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-10 22:45:36,618	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-10_22-45-36_11704/logs.
2019-06-10 22:45:36,620	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.105',
 'redis_address': '192.168.3.105:41351',
 'object_store_address': '/tmp/ray/session_2019-06-10_22-45-36_11704/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-10_22-45-36_11704/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import games_configurations

a2c_config = games_configurations.bipedalwalker_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = a2c_games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
agent = A2CAgent(sess,'walker', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-06-10 22:45:37,706	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=11814) 
(pid=11814) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=11814) For more information, please see:
(pid=11814)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=11814)   * https://github.com/tensorflow/addons
(pid=11814) If you depend on functionality not listed there, please file an issue.
(pid=11814) 
(pid=11789) 
(pid=11789) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=11789) For more information, please see:
(pid=11789)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=11789)   * https://github.com/tensorflow/addons
(pid=11789) If you depend on functionality not listed there, please file an issue.
(pid=11789) 
(pid=11793) 
(pid=11793) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=11793) For more information, please see:
(pid=11793)   * https://github.com/tensorflow/commun

(pid=11824) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=11824)   result = entry_point.load(False)
(pid=11824) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=11824) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=11830) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=11830) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=11830) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=11830)   result = entry_point.load(False)
(pid=11833) /home/trrrrr/anaconda3/envs/rl/lib/p

Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/ray/worker.py", line 1547, in print_logs
    msg = pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/trrrrr/anaconda3/envs/rl/lib/py

KeyboardInterrupt: 

In [6]:
ray.shutdown()

In [ ]:
import numpy as np
#agent.save('nn/latest')
#agent.restore('nn/carRoboschoolAnt-v1')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = True
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env.close()




In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()